In [2]:
import numpy as np
import pandas as pd
import os

In [4]:
PATH = 'C:/Workspace/python/Data_Science/dataA/ProDS특강/'
file_list = os.listdir(PATH)
file_list

['Dataset.zip',
 'Dataset_01.csv',
 'Dataset_02.csv',
 'Dataset_03.csv',
 'Dataset_04.csv',
 'Dataset_05.csv',
 'Dataset_05_item_list.csv',
 'Dataset_05_Mart_POS.csv',
 'Dataset_06.csv',
 'Dataset_07.csv',
 'Dataset_08.csv',
 'Dataset_09.csv',
 'Dataset_10.csv',
 'Dataset_11.csv',
 'Dataset_12.csv',
 'Dataset_13_test.csv',
 'Dataset_13_train.csv',
 'Dataset_14.csv',
 'Dataset_15_item_list.csv',
 'Dataset_15_Mart_POS.csv']

In [7]:
df_01 = pd.read_csv(PATH + file_list[1])
df_01.head()

,TV,Radio,Social_Media,Influencer,Sales
0,16.0,6.566231,2.907983,Mega,54.732757
1,13.0,9.237765,2.409567,Mega,46.677897
2,41.0,15.886446,2.913410,Mega,150.177829
3,83.0,30.020028,6.922304,Mega,298.246340
4,15.0,8.437408,1.405998,Micro,56.594181


In [40]:
# 총 결측값 : 26개
print(df_01.isnull().sum().sum())
# TV와 매출의 양의 상관관계가 있으며, 제일 높음.
q1 = df_01.corr().abs().drop('Sales')['Sales']
print(q1)
print(round(q1.max(), 5))
q1.argmax() # 최댓값의 위치정보
q1.idxmax() # 최댓값의 id값
q1.nlargest(1) # 최댓값의 id값과 최댓값 출력
q1.argmin() # 최솟값의 위치정보
q1.idxmin() # 최솟값의 위치정보
q1.nsmallest(1) # 최솟값의 위치정보

26
TV              0.999497
Radio           0.869105
Social_Media    0.528906
Name: Sales, dtype: float64
0.9995


Social_Media    0.528906
Name: Sales, dtype: float64

In [77]:
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols
from statsmodels.api import OLS, add_constant

q2 = df_01.dropna().copy()
x_list = ['TV', 'Radio', 'Social_Media']
lm = LinearRegression(fit_intercept=True).fit(q2[x_list], q2.Sales)
print(lm.coef_)

form1 = 'Sales~' + '+'.join(x_list) # 앞쪽변수가 종속변수, 뒤쪽 변수가 독립변수
print(form1)
# ols1 = ols(form1, data = q2) # 분리를 해줄 거면 변수를 따로 선언, 결과값만 볼꺼면 바로 fit을 해줘도 됨.
# ols2 = ols1.fit()
# dir(ols2)
ols1 = ols(form1, data = q2).fit()
# ols1.summary()
(ols1.outlier_test()['unadj_p'] < 0.05).sum()
q2[ols1.outlier_test()['unadj_p'] < 0.05]

[ 3.56256963 -0.00397039  0.00496402]
Sales~TV+Radio+Social_Media


,TV,Radio,Social_Media,Influencer,Sales
93,72.0,25.406596,3.215455,Micro,262.401171
130,21.0,1.380311,5.286681,Mega,80.940725
136,88.0,26.009640,1.495601,Nano,321.046689
162,39.0,4.856123,0.874163,Macro,131.077255
179,64.0,23.081488,3.056298,Macro,219.568184
...,...,...,...,...,...
4433,50.0,22.728177,3.794120,Mega,184.040212
4448,82.0,28.920430,3.213067,Micro,285.392143
4452,36.0,15.393056,3.601162,Nano,119.220715
4477,61.0,20.789544,2.473938,Macro,225.822151


In [81]:
df_02 = pd.read_csv(PATH + file_list[3])
df_02.columns

Index(['long_hair', 'forehead_width_cm', 'forehead_height_cm', 'nose_wide',
       'nose_long', 'lips_thin', 'distance_nose_to_lip_long', 'gender'],
      dtype='object')

In [104]:
var_list = ['forehead_width_cm', 'forehead_height_cm']
df_02_01 = df_02[var_list].copy()
df_02_01['forehead_ratio'] = df_02_01[var_list[0]] / df_02_01[var_list[1]]
forehead_ratio_avg = df_02_01['forehead_ratio'].sum() / len(df_02_01)
df_02_01['forehead_ratio_residual'] = [df_02_01['forehead_ratio'][i] - forehead_ratio_avg for i in range(len(df_02_01))]
df_02_01['forehead_ratio_residual_square'] = [(df_02_01['forehead_ratio_residual'][i]**2) for i in range(len(df_02_01))]
forehead_ratio_variance = df_02_01['forehead_ratio_residual_square'].sum() / len(df_02_01)
forehead_ratio_variance**(1/2)

0.26449884843487076

In [121]:
var_list = ['forehead_width_cm', 'forehead_height_cm']
df_02_01 = df_02[var_list].copy()
df_02_01['forehead_ratio'] = df_02_01[var_list[0]] / df_02_01[var_list[1]]
xbar = (df_02_01['forehead_ratio']).mean()
std = (df_02_01['forehead_ratio']).std()
print(xbar, std)
df_02_01[(df_02_01['forehead_ratio']) < xbar - (3*std)].sum() + df_02_01[(df_02_01['forehead_ratio']) > xbar + (3*std)]

2.233618718921447 0.26452529699735255


,forehead_width_cm,forehead_height_cm,forehead_ratio
1641,15.5,5.1,3.039216
1817,15.5,5.1,3.039216
4948,15.5,5.1,3.039216


In [147]:
print(df_02.gender.unique())
df_02['forehead_ratio'] = df_02['forehead_width_cm'] / df_02['forehead_height_cm']
gr_A = df_02[df_02.gender == 'Male']['forehead_ratio']
gr_B = df_02[df_02.gender == 'Female']['forehead_ratio']

from scipy.stats import bartlett, ttest_1samp, ttest_ind, ttest_rel
## ttest-1samp 일표본
## ttest-ind 독립인 이표본
## ttest-rel 대응인 이표본

bart1 = bartlett(gr_A, gr_B)
print(bart1.pvalue) ## p-value가 0.05미만 귀무가설(분산에 차이가 없다) 기각 -> 이분산이다.

df_02_out = ttest_ind(gr_A, gr_B, equal_var = False)
print(f'df_02_out.statistic : {df_02_out.statistic} / df_02_out.pvalue : {df_02_out.pvalue}')

round(abs(df_02_out.statistic), 3) # 3번째자리까지 남김(4번째자리에서 반올림)
print(np.ceil(df_02_out.statistic * 1000000) / 1000000) # 올림 : 정수형태로 반환하기에 반드시 곱하기 후 같은 수로 나누어 줘야함
print(np.floor(df_02_out.statistic * 1000000) / 1000000) # 내림 : 정수형태로 반환하기에 반드시 곱하기 후 같은 수로 나누어 줘야함

['Male' 'Female']
2.4617792693952707e-48
df_02_out.statistic : 2.9994984197511543 / df_02_out.pvalue : 0.0027186702390657176
2.999499
2.999498


In [186]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

df_02 = pd.read_csv(PATH + file_list[3])
x_list = ['long_hair', 'forehead_width_cm', 'forehead_height_cm', 'nose_wide','nose_long', 'lips_thin', 'distance_nose_to_lip_long']
y_list = ['gender']
df_02[y_list].value_counts()
gender_dict = dict({'Male' : 0, 'Female' : 1})
df_02['gender'] = df_02['gender'].apply(lambda x: gender_dict[x]) 
x_train, x_test, y_train, y_test = train_test_split(df_02[x_list], df_02[y_list], train_size = 0.7, test_size = 0.3, random_state = 123)
model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

In [197]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

x_list = ['long_hair', 'forehead_width_cm', 'forehead_height_cm', 'nose_wide','nose_long', 'lips_thin', 'distance_nose_to_lip_long']
df_02 = pd.read_csv(PATH + file_list[3])
df_train, df_test = train_test_split(df_02, train_size = 0.7, test_size = 0.3, random_state = 123)
model = LogisticRegression().fit(df_train[x_list], df_train.gender)
model.predict(df_test[x_list]) # 클래스명으로반환
model.predict_proba(df_test[x_list]) # 가능성으로 반환

from sklearn.metrics import precision_score, classification_report
precision_score(df_test.gender, model.predict(df_test[x_list]), pos_label = 'Male') # 0.9596354166666666
print(classification_report(df_test.gender, model.predict(df_test[x_list])))

              precision    recall  f1-score   support

      Female       0.97      0.96      0.96       743
        Male       0.96      0.97      0.97       758

    accuracy                           0.97      1501
   macro avg       0.97      0.97      0.97      1501
weighted avg       0.97      0.97      0.97      1501



In [31]:
df_03 = pd.read_csv(PATH + file_list[4])
df_03_kor = df_03[df_03['LOCATION'] == 'KOR'].copy()
round(pd.pivot_table(df_03_kor, index = 'TIME', values = 'Value', aggfunc = 'sum').reset_index().corr().TIME[1], 2)

0.96

In [71]:
df_03_KJ = df_03[df_03.LOCATION.isin(['KOR', 'JPN'])]
subject_list = df_03_KJ.SUBJECT.unique()

# 대응 t테스트 H0 : 평균차이가 없다. H1 : 평균차이가 있다.
from scipy.stats import ttest_rel
ttest_sample = pd.pivot_table(df_03_KJ[df_03_KJ.SUBJECT == 'BEEF'], index = 'TIME', columns = 'LOCATION', values = 'Value').dropna()
ttest_out = ttest_rel(ttest_sample.KOR, ttest_sample.JPN)
p_value = ttest_out.pvalue

subject_dict = dict()
target_dict = dict()
for SUBJECT in subject_list:
    ttest_sample = pd.pivot_table(df_03_KJ[df_03_KJ.SUBJECT == SUBJECT], index = 'TIME', columns = 'LOCATION', values = 'Value').dropna()
    ttest_out = ttest_rel(ttest_sample.KOR, ttest_sample.JPN)
    p_value = ttest_out.pvalue
    subject_dict[SUBJECT] = p_value
    if p_value >= 0.05:
        target_dict[SUBJECT] = p_value
target_dict

{'POULTRY': 0.4690625508910583}

In [134]:
df_03_kor = df_03[df_03.LOCATION == 'KOR']
subject_list = df_03_kor.SUBJECT.unique()

from sklearn.linear_model import LinearRegression
temp = df_03_kor[df_03_kor.SUBJECT == 'BEEF']
lm = LinearRegression().fit(temp[['TIME']], temp['Value']) # 반드시 2차식 구조로 넣어주어야 함.
pred = lm.predict(temp[['TIME']])
r2_score = lm.score(temp[['TIME']], temp[['Value']])

# coef_dict = dict()
# for SUBJECT in subject_list:
#     temp = df_03_kor[df_03_kor.SUBJECT == SUBJECT]
#     lm = LinearRegression().fit(temp[['TIME']], temp[['Value']]) # 반드시 2차식 구조로 넣어주어야 함.
#     coef_dict[SUBJECT] = lm.coef_[0][0]
# coef_dict

mape_dict = dict()
for SUBJECT in subject_list:
    temp = df_03_kor[df_03_kor.SUBJECT == SUBJECT]
    lm = LinearRegression().fit(temp[['TIME']], temp['Value']) # x값은 반드시 1차원으로 넣을 수 없음. y값은 상관없음
    pred = lm.predict(temp[['TIME']])
    r2_score = lm.score(temp[['TIME']], temp[['Value']])
    mape = (abs(temp['Value'] - pred) / temp['Value']).sum() * 100 / len(temp)
    mape_dict[SUBJECT] = mape, r2_score
mape_dict
pd.DataFrame(mape_dict,index = ['mape', 'r2_score']).T

## 번외
for SUBJECT in subject_list:
    temp = df_03_kor[df_03_kor.SUBJECT == SUBJECT]
    globals()['lm_'+ SUBJECT] = LinearRegression().fit(temp[['TIME']], temp['Value']) # x값은 반드시 1차원으로 넣을 수 없음. y값은 상관없음
    pred = eval('lm'+ SUBJECT).predict(temp[['TIME']])
    r2_score = lm.score(temp[['TIME']], temp[['Value']])
    mape = (abs(temp['Value'] - pred) / temp['Value']).sum() * 100 / len(temp)
    mape_dict[SUBJECT] = mape, r2_score
# lm_BEEF의 방식으로 모델이 모두 저장되어 있음. 

LinearRegression()